In [17]:
rawData = sc.textFile('case-control-equal.txt')
print rawData.first()
print rawData.count()


2135579a4aa630,1,"2135579a4aa630 1713684 71fe092b72aa56be5998fbf23e6a826a 0 1 0 0 0 1659495.0 2 Gewijzigd 1 1 x per dag 20 milligram ORAAL 0 20.0 1 MILLIGRAM 1 D 2012-02-07 2012-01-23 OLANZAPINE SMELTTABLET 20MG 0 SMELTTABLET N05AH03 PSYCHOLEPTICS NERVOUS SYSTEM Diazepines, oxazepines and thiazepines ANTIPSYCHOTICS 20.0 olanzapine 1 - Tuesday 4804 15479 0 - Monday 0 1 1 x per dag 20 milligram 2135579a4aa630 4804 2011-12-15 2012-02-06 Klinische opname Klinische behandeling 53.0 3 - Thursday 0 - Monday False False 25.0 18-30 False KL. PSYCHOTISCHE STOORNISSEN 1 True"
2454


In [18]:
tuples = rawData.map(lambda l: l.split(",")).map(lambda a : (str(a[0]), int(a[1]), str(a[2].encode('utf8'))))
print tuples.take(3)
print tuples.count()

[('2135579a4aa630', 1, '"2135579a4aa630 1713684 71fe092b72aa56be5998fbf23e6a826a 0 1 0 0 0 1659495.0 2 Gewijzigd 1 1 x per dag 20 milligram ORAAL 0 20.0 1 MILLIGRAM 1 D 2012-02-07 2012-01-23 OLANZAPINE SMELTTABLET 20MG 0 SMELTTABLET N05AH03 PSYCHOLEPTICS NERVOUS SYSTEM Diazepines'), ('dbd484faf711a0', 1, '"dbd484faf711a0 1693355 71fe092b72aa56be5998fbf23e6a826a 0 1 0 0 0 1693355.0 1 Initiele verstrekking 1 1 x per dag 2 milligram ORAAL 0 2.0 1 MILLIGRAM 1 D 2012-01-17 2012-01-16 RISPERIDON SMELTTABLET 2MG 0 SMELTTABLET N05AX08 PSYCHOLEPTICS NERVOUS SYSTEM Other antipsychotics ANTIPSYCHOTICS 2.0 risperidon 1 - Tuesday -1 14554 0 - Monday 0 1 1 x per dag 2 milligram"'), ('7ce58679a76272', 1, '"7ce58679a76272 2012-01-05 2012-06-30 as1_6.02.01.03.04 4463 Bipolaire I stoornis: Laatste episode manisch')]
2454


In [19]:
from pyspark.mllib.feature import HashingTF
htf = HashingTF()

In [20]:
hashedTuples = tuples.map(lambda (i, c, d) : (i, c, htf.transform(d)))

In [21]:
print hashedTuples.first()

('2135579a4aa630', 1, SparseVector(1048576, {8148: 1.0, 19142: 3.0, 30136: 6.0, 60275: 2.0, 71269: 2.0, 82263: 1.0, 93257: 2.0, 134386: 1.0, 145380: 7.0, 167368: 4.0, 189356: 4.0, 219495: 3.0, 263471: 2.0, 274465: 36.0, 293606: 3.0, 304600: 2.0, 378715: 1.0, 389709: 9.0, 411697: 20.0, 422691: 1.0, 430838: 1.0, 441832: 6.0, 463820: 6.0, 485808: 13.0, 504953: 1.0, 515947: 1.0, 537935: 4.0, 559923: 15.0, 612046: 4.0, 623040: 8.0, 634034: 6.0, 653179: 2.0, 664173: 2.0, 686161: 3.0, 697155: 2.0, 708149: 3.0, 738284: 2.0, 771266: 2.0, 782260: 7.0, 812399: 1.0, 834387: 3.0, 845381: 2.0, 856375: 6.0, 897504: 10.0, 908498: 7.0, 919492: 8.0, 930486: 5.0, 960625: 2.0, 971619: 3.0, 982613: 8.0, 1004601: 3.0}))


In [22]:
from pyspark.mllib.regression import LabeledPoint
inputData = hashedTuples.map(lambda (i,c,v): LabeledPoint(c,v))

In [23]:
training, testing, validation = inputData.randomSplit([0.7,0.3,0.0], 5)

In [24]:
print training.first()

(1.0,(1048576,[8148,19142,30136,60275,71269,82263,93257,134386,145380,167368,189356,208501,219495,230489,263471,274465,282612,293606,304600,356727,367721,389709,411697,422691,441832,463820,485808,504953,537935,559923,579064,590058,612046,623040,634034,664173,686161,697155,708149,738284,771266,782260,812399,834387,845381,856375,886510,897504,908498,919492,930486,960625,971619,982613,1004601,1045730],[1.0,2.0,5.0,9.0,7.0,1.0,2.0,5.0,12.0,8.0,7.0,5.0,5.0,1.0,3.0,57.0,1.0,5.0,3.0,1.0,2.0,8.0,19.0,2.0,11.0,7.0,22.0,2.0,4.0,13.0,3.0,2.0,6.0,7.0,5.0,5.0,4.0,2.0,4.0,4.0,4.0,9.0,5.0,5.0,2.0,6.0,3.0,13.0,10.0,8.0,4.0,4.0,4.0,11.0,4.0,2.0]))


In [25]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing.count()



In [34]:
tp = float(predictionAndLabel.filter(lambda (x, v): x == 1 and x == v).count())
fp =  float(predictionAndLabel.filter(lambda (x, v): x == 1 and x != v).count())
precision = tp / (tp + fp)
print precision

0.842105263158


In [29]:
print accuracy

0.876984126984


In [30]:
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
count = testing.filter(lambda p: p.label == 0).count()

accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v and v == 0).count() / count
print accuracy

0.838461538462


In [31]:
predictionAndLabel.filter(lambda (x, v): x == 0).count()

357

In [32]:
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
count = testing.filter(lambda p: p.label == 1).count()

accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v and v == 1).count() / count
print accuracy

0.918032786885


In [33]:
predictionAndLabel.filter(lambda (x, v): x == 1).count()

399